
# MotionDataset2label with ResNet

In [1]:
%install-location /home/michal/DeepLearning/language2motion
%install-swiftpm-flags -c release
// %install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets ImageClassificationModels
%install '.package(path: "/home/michal/DeepLearning/language2motion/code")' Batcher ModelSupport Datasets ImageClassificationModels


Installing packages:
	.package(path: "/home/michal/DeepLearning/language2motion/code")
		Batcher
		ModelSupport
		Datasets
		ImageClassificationModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp28ve2luz/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import PythonKit

import Batcher
import ModelSupport
import Datasets
import ImageClassificationModels

In [3]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


# Checking X10 warmup

In [4]:
let date = Date()
date

▿ 2020-06-03 22:55:11 +0000
  - timeIntervalSinceReferenceDate : 612917711.447093


In [5]:
let eagerTensor1 = Tensor([0.0, 1.0, 2.0])
let eagerTensor2 = Tensor([1.5, 2.5, 3.5])
let eagerTensorSum = eagerTensor1 + eagerTensor2
eagerTensorSum

[1.5, 3.5, 5.5]


In [6]:
eagerTensor1.device

▿ Device(kind: .CPU, ordinal: 0, backend: .TF_EAGER)
  - kind : TensorFlow.Device.Kind.CPU
  - ordinal : 0
  - backend : TensorFlow.Device.Backend.TF_EAGER


In [7]:
let x10Tensor2 = Tensor([1.5, 2.5, 3.5], on: Device.defaultXLA)
x10Tensor2.device

▿ Device(kind: .GPU, ordinal: 0, backend: .XLA)
  - kind : TensorFlow.Device.Kind.GPU
  - ordinal : 0
  - backend : TensorFlow.Device.Backend.XLA


# load dataset

In [8]:
let batchSize = 25

In [9]:


let serializedDatasetURL = URL(fileURLWithPath: "/home/michal/DeepLearning/language2motion/data/motion_dataset_v1.plist")
let labelsURL = URL(fileURLWithPath: "/home/michal/DeepLearning/language2motion/data/labels_ds_v2.csv")

let dataset = Motion2Label(
    batchSize: batchSize, 
    serializedDatasetURL: serializedDatasetURL,
    labelsURL: labelsURL
)
print("dataset.training.count: \(dataset.training.count)")
print("dataset.test.count: \(dataset.test.count)")

MotionData(motionSamples: 3911)
trainTensorPairs.count = 2410
testTensorPairs.count = 602
dataset.training.count: 97
dataset.test.count: 25


In [10]:
// Statistics from X10 example
struct Statistics {
    var correctGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalGuessCount = Tensor<Int32>(0, on: Device.default)
    var totalLoss = Tensor<Float>(0, on: Device.default)
    var batches: Int = 0
    var accuracy: Float { 
        Float(correctGuessCount.scalarized()) / Float(totalGuessCount.scalarized()) * 100 
    } 
    var averageLoss: Float { totalLoss.scalarized() / Float(batches) }

    init(on device: Device = Device.default) {
        correctGuessCount = Tensor<Int32>(0, on: device)
        totalGuessCount = Tensor<Int32>(0, on: device)
        totalLoss = Tensor<Float>(0, on: device)
    }

    mutating func update(logits: Tensor<Float>, labels: Tensor<Int32>, loss: Tensor<Float>) {
        let correct = logits.argmax(squeezingAxis: 1) .== labels
        correctGuessCount += Tensor<Int32>(correct).sum()
        totalGuessCount += Int32(labels.shape[0])
        totalLoss += loss
        batches += 1
    }
}

# 1-channel ResNet

# Eager

In [11]:
var eagerModel = ResNet(classCount: 5, depth: .resNet18, downsamplingInFirstStage: false, channelCount: 1 )
var eagerOptimizer = SGD(for: eagerModel, learningRate: 0.001)

In [12]:
//Eager

print("Starting Eager training...")

for epoch in 1...5 {
    let start = Date()
    var trainStats = Statistics()
    var testStats = Statistics()
    
    Context.local.learningPhase = .training
    for batch in dataset.training.sequenced() {
        let (images, labels) = (batch.first, batch.second)
        let 𝛁model = TensorFlow.gradient(at: eagerModel) { eagerModel -> Tensor<Float> in
            let ŷ = eagerModel(images)
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }
        eagerOptimizer.update(&eagerModel, along: 𝛁model)
    }

    Context.local.learningPhase = .inference
    for batch in dataset.test.sequenced() {
        let (images, labels) = (batch.first, batch.second)
        let ŷ = eagerModel(images)
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}

Starting Eager training...
[Epoch 1] Training Loss: 1.398, Training Accuracy: 1028/2410 (42.7%), Test Loss: 1.278, Test Accuracy: 322/602 (53.5%) seconds per epoch: 26.3
[Epoch 2] Training Loss: 1.191, Training Accuracy: 1299/2410 (53.9%), Test Loss: 1.199, Test Accuracy: 325/602 (54.0%) seconds per epoch: 23.8
[Epoch 3] Training Loss: 1.129, Training Accuracy: 1357/2410 (56.3%), Test Loss: 1.151, Test Accuracy: 332/602 (55.1%) seconds per epoch: 23.8
[Epoch 4] Training Loss: 1.084, Training Accuracy: 1394/2410 (57.8%), Test Loss: 1.121, Test Accuracy: 337/602 (56.0%) seconds per epoch: 23.7
[Epoch 5] Training Loss: 1.054, Training Accuracy: 1420/2410 (58.9%), Test Loss: 1.095, Test Accuracy: 353/602 (58.6%) seconds per epoch: 24.0


# X10

In [13]:
let device = Device.defaultXLA
device

▿ Device(kind: .GPU, ordinal: 0, backend: .XLA)
  - kind : TensorFlow.Device.Kind.GPU
  - ordinal : 0
  - backend : TensorFlow.Device.Backend.XLA


In [14]:
//var model = ResNet(classCount: 5, depth: .resNet18, downsamplingInFirstStage: false, channelCount: 1)
var model = ResNet(classCount: 5, depth: .resNet18, downsamplingInFirstStage: false, channelCount: 1 )

model.move(to: device)

In [15]:
var optimizer = SGD(for: model, learningRate: 0.001)
optimizer = SGD(copying: optimizer, to: device)

In [16]:
// X10 training
print("Starting X10 training...")
for epoch in 1...5 {
    let start = Date()
    var trainStats = Statistics(on: device)
    var testStats = Statistics(on: device)
    
    Context.local.learningPhase = .training
    for batch in dataset.training.sequenced() {
        let (eagerImages, eagerLabels) = (batch.first, batch.second)
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: eagerLabels, to: device)
        let 𝛁model = TensorFlow.gradient(at: model) { model -> Tensor<Float> in
            let ŷ = model(images)
            let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
            trainStats.update(logits: ŷ, labels: labels, loss: loss)
            return loss
        }
        optimizer.update(&model, along: 𝛁model)
        LazyTensorBarrier()
    }

    Context.local.learningPhase = .inference
    for batch in dataset.test.sequenced() {
        let (eagerImages, eagerLabels) = (batch.first, batch.second)
        let images = Tensor(copying: eagerImages, to: device)
        let labels = Tensor(copying: eagerLabels, to: device)
        let ŷ = model(images)
        let loss = softmaxCrossEntropy(logits: ŷ, labels: labels)
        LazyTensorBarrier()
        testStats.update(logits: ŷ, labels: labels, loss: loss)
    }

    print(
        """
        [Epoch \(epoch)] \
        Training Loss: \(String(format: "%.3f", trainStats.averageLoss)), \
        Training Accuracy: \(trainStats.correctGuessCount)/\(trainStats.totalGuessCount) \
        (\(String(format: "%.1f", trainStats.accuracy))%), \
        Test Loss: \(String(format: "%.3f", testStats.averageLoss)), \
        Test Accuracy: \(testStats.correctGuessCount)/\(testStats.totalGuessCount) \
        (\(String(format: "%.1f", testStats.accuracy))%) \
        seconds per epoch: \(String(format: "%.1f", Date().timeIntervalSince(start)))
        """)
}

Starting X10 training...
[Epoch 1] Training Loss: 1.382, Training Accuracy: 1092/2410 (45.3%), Test Loss: 1.262, Test Accuracy: 315/602 (52.3%) seconds per epoch: 37.7
[Epoch 2] Training Loss: 1.186, Training Accuracy: 1331/2410 (55.2%), Test Loss: 1.171, Test Accuracy: 325/602 (54.0%) seconds per epoch: 16.7
[Epoch 3] Training Loss: 1.111, Training Accuracy: 1388/2410 (57.6%), Test Loss: 1.120, Test Accuracy: 336/602 (55.8%) seconds per epoch: 8.8
[Epoch 4] Training Loss: 1.054, Training Accuracy: 1424/2410 (59.1%), Test Loss: 1.085, Test Accuracy: 338/602 (56.1%) seconds per epoch: 8.7
[Epoch 5] Training Loss: 1.026, Training Accuracy: 1442/2410 (59.8%), Test Loss: 1.065, Test Accuracy: 354/602 (58.8%) seconds per epoch: 8.8
